In [8]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import math

In [9]:
data=pd.read_excel('Melbourn_Input.xls', sheet_name=1)
data_with_misiing=pd.read_excel('Melbourn_Input.xls', sheet_name=0)

In [10]:
# a few records are replaced with mean of the column
data_with_misiing['Ammonia_mg_L_1'].fillna(data_with_misiing['Ammonia_mg_L_1'].mean(), inplace=True)
data_with_misiing['COD_mg_L_1'].fillna(data_with_misiing['COD_mg_L_1'].mean(), inplace=True)
for i in range(len(data_with_misiing.index)):
    if str(data_with_misiing.iloc[i,4])=='nan':
        data_with_misiing.iloc[i,4]=data_with_misiing.iloc[i,5]/2

In [11]:
data_with_misiing.isna().sum()

recorddate                      0
feeder1_kwh                     0
feeder2_kwh                     0
Ammonia_mg_L_1                  0
BOD_mg_L_1                      0
COD_mg_L_1                      0
NitrateplusNitrite_mg_L_1    1244
Nitrogentotal_mg_L_1         1244
Day                             0
T                               0
TM                              0
Tm                              0
SLP                             0
H                               0
PP                              0
VV                              0
V                               0
VM                              0
VG                              0
dtype: int64

In [13]:
def clean_data(data):
    
    #Droping irrelevant columns
    extra_columns=['NitrateplusNitrite_mg_L_1', 'Day']
    data=data.drop(columns=extra_columns)
    
    #using the record date column
    date=pd.to_datetime(data['recorddate'])
    data['year']=date.dt.year
    data['month']=date.dt.month
    data['day']=date.dt.day
    #droping the recorddate
    data=data.drop(columns=['recorddate'])
    
    #normalizing data
    scaler = MinMaxScaler()
    scaler.fit(data)
    normData = pd.DataFrame(scaler.transform(data.values), index=data.index, columns=data.columns)
    
    return scaler,normData

In [14]:
def train_GBM(x,y):
    from sklearn.ensemble import GradientBoostingRegressor
    x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=42)
    GB=GradientBoostingRegressor(n_estimators=1000,learning_rate=0.001
                                ,min_samples_leaf=8 ,min_samples_split=10, max_depth=3)
    GB.fit(x_train,y_train)
    y_pred_GBM=GB.predict(x_test)
    y_pred_GBM2=GB.predict(x)
    print ( 'R2_GBM_Test= ',R2(y_test,y_pred_GBM.flatten()) )
    print ( 'R2_GBM_ALL= ',R2(y, y_pred_GBM2.flatten()) )
    return GB

In [15]:
def R2(y_true, y_pred):
    #Calculation of the unadjusted r-squared, goodness of fit metric
    sse  = np.square( y_pred - y_true ).sum()
    sst  = np.square( y_true - y_true.mean() ).sum()
    return 1 - sse/sst

In [16]:
scaler,normdata=clean_data(data)
y=normdata['Nitrogentotal_mg_L_1']
x=normdata.drop(columns=['Nitrogentotal_mg_L_1'])
GB=train_GBM(x,y)


R2_GBM_Test=  0.45170658817591025
R2_GBM_ALL=  0.5325962403843114


In [17]:
new_scaler,new_normdata=clean_data(data_with_misiing)

In [18]:
# Replacing the missing values with the predicted values
TN_pred=GB.predict(new_normdata.drop(columns=['Nitrogentotal_mg_L_1']))
for i in range(len(new_normdata.index)):
    if str(new_normdata.iloc[i,5])=='nan':
        new_normdata.iloc[i,5]=TN_pred[i]

In [19]:
inv_scaled=pd.DataFrame(new_scaler.inverse_transform(new_normdata.values),
                        index=new_normdata.index, columns=new_normdata.columns)

In [20]:
inv_scaled.to_csv('CleanData.csv')

In [23]:
inv_scaled

,feeder1_kwh,feeder2_kwh,Ammonia_mg_L_1,BOD_mg_L_1,COD_mg_L_1,Nitrogentotal_mg_L_1,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,year,month,day
0,105695.0,70161.0,27.0,365.0,730.0,60.377845,19.3,25.1,12.6,0.0,56.0,1.52,10.0,26.9,53.5,79.5,2014.0,1.0,1.0
1,111836.0,69788.0,25.0,370.0,740.0,60.026153,17.1,23.6,12.3,0.0,63.0,0.00,10.0,14.4,27.8,0.0,2014.0,1.0,2.0
2,140088.0,61928.0,42.0,418.0,836.0,64.522311,16.8,27.2,8.8,0.0,47.0,0.25,10.0,31.9,61.1,77.8,2014.0,1.0,5.0
3,140121.0,67426.0,36.0,430.0,850.0,63.000000,14.6,19.9,11.1,0.0,49.0,0.00,10.0,27.0,38.9,53.5,2014.0,1.0,6.0
4,135260.0,67564.0,46.0,508.0,1016.0,65.590267,13.4,19.1,8.0,0.0,65.0,0.00,10.0,20.6,35.2,53.5,2014.0,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,167475.0,138463.0,28.0,310.0,660.0,58.806985,6.8,10.0,4.0,0.0,56.0,0.00,10.0,14.4,22.2,0.0,2019.0,6.0,20.0
1378,160476.0,132970.0,40.0,250.0,700.0,60.670725,7.6,16.0,3.0,0.0,71.0,0.00,0.0,13.5,25.9,0.0,2019.0,6.0,24.0
1379,174179.0,133789.0,47.0,260.0,860.0,64.000000,9.7,16.0,3.0,0.0,65.0,0.00,10.0,18.1,25.9,0.0,2019.0,6.0,25.0
1380,166213.0,150462.0,46.0,300.0,900.0,62.478549,10.9,17.0,5.0,0.0,65.0,0.00,10.0,21.9,35.2,51.9,2019.0,6.0,26.0
